# Introduction
This notebook shows how to create a new policy. There are two ways to do so:

1. Extend the `DefaultPolicy`: this approach is mostly suitable if you just require some minor changes to the defautl behavior, like creating different order types.
2. Implement the `Policy` from scratch: this approach provides full flexibility on what the policy should do.

In [ ]:
%use @http://roboquant.org/roboquant.json
Welcome()

## Extending the `DefaulPolicy`

We're now ready to develop a "custom policy". For this first example we extend the DefaultPolicy and create our own Order type by overriding the `createOrder` method. The default behavior is to create a `MarketOrder`, but we change it to a `BracketOrder`.

In [ ]:
class MyPolicy(var percentage: Double=0.05) : DefaultPolicy() {

        override fun createOrder(signal: Signal, qty: Double, price: Double): Order? {
            
            // I this example we don't short and exit orders are covered by the bracket order.
            if (qty < 0) return null
            
            val asset = signal.asset
      
            return BracketOrder(
                MarketOrder(asset, qty),
                TrailOrder(asset, -qty, percentage/2.0),
                StopOrder(asset, -qty, price* (1 - percentage))
            )
        }
}

## Create a new roboquant and evaluate it
We are now ready to back-test our strategy using the just created Policy. And suppose we want to find out what is the optimal percentage to use in our policy, we'll iterate over 10 values to see what results in the best solution.

In [ ]:
val feed = AvroFeed.sp500()
val strategy = EMACrossover()
val policy = MyPolicy()
val roboquant =  Roboquant(strategy, AccountSummary(), policy = policy)

for (percentage in 1..10) {
    policy.percentage = percentage / 100.0
    roboquant.run(feed, runName = "run-$percentage%")
}

In [ ]:
var data = roboquant.logger.getMetric("account.equity")
MetricChart(data)

In [ ]:
data.max()

# Implementing the `Policy` interface